## Using the RNC to look at preps in certain windows of verbs

In [1]:
import random

In [2]:
from corus import load_morphoru_rnc

path = 'RNCgoldInUD_Morpho.conll'
records = load_morphoru_rnc(path)
rnccorpus = []
for record in records:
    rnccorpus.append(record)
len(rnccorpus)

98892

## prep + case!!!

In [3]:
# def build_sentence(tokens, upperindexlist):
#     sentence = ''
#     for i, token in enumerate(tokens):
#         if token.pos == 'PUNCT':
#             space = ''
#         else:
#             space = ' '
#         if i in upperindexlist:
#             text = token.text.upper()
#         else:
#             text = token.text
#         if text == None:
#             text = ''
#         sentence = sentence + space + text
#     return sentence.strip()

In [69]:
for i, sent in enumerate(rnccorpus):
    formattedsent = build_sentence(sent.tokens,[])
    if 'инструмент ПОСАДИТ ВАС на колени к черно-белым клавишам'.lower() in formattedsent.lower():
        thesetokens = sent
        index = i
        break

In [70]:
formattedsent

'- Дома, конечно, был инструмент Посадит вас на колени к черно-белым клавишам...'

In [71]:
index

35557

In [124]:
def build_sentence(tokens, upperlist=[]):
    sentence = ''
    for i, token in enumerate(tokens):
        if token.pos == 'PUNCT':
            space = ''
        else:
            space = ' '
        if i in upperlist:
            text = token.text.upper()
        else:
            text = token.text
        if text == None:
            text = ''
        sentence = sentence + space + text
    return sentence.strip()
adpdict = {
    'в' : 'в/во',
    'во' : 'в/во',
    'с' : 'с/со',
    'со' : 'с/со',
    'о' : 'о/об',
    'об' : 'о/об',
    'обо' : 'о/об'
}
# sample = random.sample(rnccorpus,1000)
# sample = rnccorpus[35557:35558]
sample = rnccorpus
windowList = []
for window in range(-3, 4):
    dataDict = dict()
    for sent in sample:
        currentindex = 0
        for token in sent.tokens:
            cxfound = False
            if token.pos == 'VERB':
                verblemma = token.lemma
                dataDict.setdefault(verblemma, dict())
                dataDict[verblemma].setdefault('counts',dict())
                dataDict[verblemma].setdefault('sentences',dict())
                
                adpindex = currentindex + window
                if adpindex >= 0 and adpindex < len(sent.tokens):
                    adptoken = sent.tokens[adpindex]
                    if adptoken.pos == 'PUNCT' and adpindex + 1 >= 0 and adpindex + 1 < len(sent.tokens):
                        adpindex = adpindex + 1
                    if adptoken.pos == 'ADP':
                        adplemma = adptoken.lemma
                        # lemmatize prepositions
                        if adplemma in adpdict:
                            adplemma = adpdict[adplemma]
                        caseindex = adpindex + 1
                        if caseindex >= 0 and caseindex < len(sent.tokens):
                            casetoken = sent.tokens[caseindex]
                            if casetoken.pos == 'PUNCT' and caseindex + 1 >= 0 and caseindex + 1 < len(sent.tokens):
                                caseindex = caseindex + 1
                            casetoken = sent.tokens[caseindex]
                            try:
                                case = casetoken.feats['Case']
                                if case == 'Ins':
                                    case = 'Inst'
                                adpandcase = adplemma + ' + ' + case.upper()
                                cxfound = True    
                            except:
                                cxfound = False            
                    else:
                        if window in [-1,1]:
                            # not an adposition directly before or after verb
                            try: 
                                if adpindex - 1 >= 0:
                                    # need to make sure prev token is not adp
                                    if sent.tokens[adpindex - 1].pos != 'ADP':
                                        case = adptoken.feats['Case']
                                        if case == 'Ins':
                                            case = 'Inst'
                                        adpandcase = case.upper()
                                        caseindex = currentindex
                                        cxfound = True    
                            except:
                                # check if infinitive, if so, set caseindex to current index
                                if adptoken.pos == 'VERB':
                                    if 'VerbForm' in adptoken.feats and (adptoken.feats['VerbForm'] == 'Inf'):
                                        adpandcase = 'INFINITIVE'
                                        caseindex = currentindex
                                        cxfound = True    
                if cxfound:
                    dataDict[verblemma]['counts'].setdefault(adpandcase, 0)
                    dataDict[verblemma]['sentences'].setdefault(adpandcase, [])
                    dataDict[verblemma]['counts'][adpandcase] += 1
                    formattedsent = build_sentence(sent.tokens, [currentindex, adpindex, caseindex])
                    if formattedsent not in dataDict[verblemma]['sentences'][adpandcase]:
                        dataDict[verblemma]['sentences'][adpandcase].append(formattedsent)
            currentindex += 1
    print(f'window size of {window} complete...')
    windowList.append(dataDict)

window size of -3 complete...
window size of -2 complete...
window size of -1 complete...
window size of 0 complete...
window size of 1 complete...
window size of 2 complete...
window size of 3 complete...


In [125]:
# removing uncommon verbs (<5 occurrences)
uncommonVerbs = set()
totalcount = dict()
for windowDict in windowList:
    for verb in windowDict:
        totalcount.setdefault(verb,0)
        for cx in windowDict[verb]['counts']:
            totalcount[verb] += windowDict[verb]['counts'][cx]
for verb in totalcount:
    if totalcount[verb] < 5:
        uncommonVerbs.add(verb)
print(len(uncommonVerbs))

5913


In [126]:
for verbtoremove in uncommonVerbs:
    for windowDict in windowList:
        try:
            del windowDict[verbtoremove]
        except:
            False

In [127]:
len(windowList[1])

3395

In [128]:
for windowDict in windowList:
    print(windowDict['избежать']['counts'])

{}
{}
{'ACC': 1, 'GEN': 4, 'NOM': 3, 'INST': 2}
{}
{'GEN': 25, 'DAT': 1}
{'в/во + LOC': 1, 'с/со + INST': 1}
{'к + DAT': 2, 'в/во + ACC': 1, 'с/со + GEN': 1}


In [84]:
pwd

'/Users/paigelee/Desktop/spring2021/clancy/verbhistograms'

## for csv

In [129]:
allcxlabels = set()
for windowDict in windowList:
    for verb in windowDict:
        countdict = windowDict[verb]['counts']
        for cxlabel in countdict:
            allcxlabels.add(cxlabel)
allcxlabels = list(allcxlabels)

In [130]:
# getting rid of cxx with count < 10 occurrences across verbs
totalcxcounts = dict()
for windowDict in windowList:
    for verb in windowDict:
        countdict = windowDict[verb]['counts']
        for cx in countdict:
            totalcxcounts.setdefault(cx, 0)
            totalcxcounts[cx] += 1
lst = []
for cx in totalcxcounts:
    lst.append((totalcxcounts[cx],cx))
i = 0
greaterthan10cx = []
for ct, cx in lst:
    if ct > 10:
        greaterthan10cx.append(cx)

In [131]:
# verb, windowsize, prep1, prep2, prep3, prep4, prep5...
rows = []
cxused = greaterthan10cx
for windowIndex, windowDict in enumerate(windowList):
    windowSize = windowIndex - 3
    for verb in windowDict:
        countdict = windowDict[verb]['counts']
        csvline = [verb, windowSize]
        for cxlabel in cxused:
            if cxlabel in countdict:
                count = countdict[cxlabel]
            else:
                count = 0
            csvline.append(count)
        rows.append(csvline)

In [132]:
r = random.choice(rows)
print(r)

['обрушиваться', 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [133]:
fields = ['Verb','WindowSize']

for cxlabel in greaterthan10cx:
    fields.append(cxlabel)

In [134]:
import csv
# writing to csv file 
filename = '8-11-21csvdata.csv'
with open(filename, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the fields 
    csvwriter.writerow(fields) 
        
    # writing the data rows 
    csvwriter.writerows(rows)

In [135]:
cd verbhistograms/

[Errno 2] No such file or directory: 'verbhistograms/'
/Users/paigelee/Desktop/spring2021/clancy/verbhistograms


In [136]:
sentenceDict = dict()
windowSize = -3
for windowDict in windowList:
    sentenceDict[windowSize] = dict()
    for verb in windowDict:
        sentenceDict[windowSize][verb] = dict()
        for cx in windowDict[verb]['sentences']:
            sentenceDict[windowSize][verb][cx] = windowDict[verb]['sentences'][cx][:10]
    windowSize += 1

In [137]:
totalsents = 0
for windowDict in windowList:
    for verb in windowDict:
        for prep in windowDict[verb]['sentences']:
            totalsents += len(windowDict[verb]['sentences'][prep])
print(totalsents)

151344


In [138]:
verbDict = dict()

for windowDict in windowList:
    for verb in windowDict:
        verbDict[verb] = dict()
windowSize = -3
for windowDict in windowList:
    for verb in windowDict:
        verbDict[verb][windowSize] = dict()
        for cx in windowDict[verb]['sentences']:
            verbDict[verb][windowSize][cx] = list()
            for sent in windowDict[verb]['sentences'][cx]:
                verbDict[verb][windowSize][cx].append(sent)
    windowSize += 1

In [140]:
import json
for verb in verbDict:
    with open(f'sentdata6/{verb}.json', 'w', encoding='utf8') as jsonfile:
        json.dump(verbDict[verb], jsonfile, ensure_ascii=False)

### get most common constructions

In [96]:
def build_sentence(tokens, upperlist):
    sentence = ''
    for i, token in enumerate(tokens):
        if token.pos == 'PUNCT':
            space = ''
        else:
            space = ' '
        if i in upperlist:
            text = token.text.upper()
        else:
            text = token.text
        if text == None:
            text = ''
        sentence = sentence + space + text
    return sentence.strip()
adpdict = {
    'в' : 'в/во',
    'во' : 'в/во',
    'с' : 'с/со',
    'со' : 'с/со',
    'о' : 'о/об',
    'об' : 'о/об',
    'обо' : 'о/об'
}
# sample = random.sample(rnccorpus,1000)
# sample = rnccorpus[10018:10025]
sample = rnccorpus

commonDict = dict()
for i, sent in enumerate(sample):
    if i % 10000 == 0:
        print(f'{i}/{len(sample)} sentences parsed...')
    currentindex = 0
    for token in sent.tokens:
        # for each verb found
        if token.pos == 'VERB':
            verblemma = token.lemma
            commonDict.setdefault(verblemma, dict())
            before = []
            after = []
            for window in range(-3,4):
                cxfound = False
                adpindex = currentindex + window

                if adpindex >= 0 and adpindex < len(sent.tokens):
                    adptoken = sent.tokens[adpindex]
                    if adptoken.pos == 'PUNCT' and adpindex + 1 >= 0 and adpindex + 1 < len(sent.tokens):
                        adpindex = adpindex + 1
                    if adptoken.pos == 'ADP':
                        adplemma = adptoken.lemma
                        # lemmatize prepositions
                        if adplemma in adpdict:
                            adplemma = adpdict[adplemma]
                        caseindex = adpindex + 1
                        if caseindex >= 0 and caseindex < len(sent.tokens):
                            casetoken = sent.tokens[caseindex]
                            if casetoken.pos == 'PUNCT' and caseindex + 1 >= 0 and caseindex + 1 < len(sent.tokens):
                                caseindex = caseindex + 1
                            casetoken = sent.tokens[caseindex]
                            try:
                                case = casetoken.feats['Case']
                                if case == 'Ins':
                                    case = 'Inst'
                                adpandcase = adplemma + ' + ' + case.upper()
                                cxfound = True    
                            except:
                                cxfound = False
                    else:
                        if window in [-1,1]:
                            # not an adposition directly before or after verb
                            try: 
                                if adpindex >= 0:
                                    # need to make sure prev token is not adp
                                    if sent.tokens[adpindex - 1].pos != 'ADP':
                                        case = adptoken.feats['Case']
                                        if case == 'Ins':
                                            case = 'Inst'
                                        adpandcase = case.upper()
                                        caseindex = currentindex
                                        cxfound = True
                            except:
                                # check if infinitive, if so, set caseindex to current index
                                if adptoken.pos == 'VERB':
                                    if 'VerbForm' in adptoken.feats and (adptoken.feats['VerbForm'] == 'Inf'):
                                        adpandcase = 'INFINITIVE'
                                        caseindex = currentindex
                                        cxfound = True
                if cxfound:
                    if window < 0:
                        before.append(adpandcase)
                    else:
                        after.append(adpandcase)
#                 dataDict[verblemma]['counts'].setdefault(adpandcase, 0)
#                 dataDict[verblemma]['sentences'].setdefault(adpandcase, [])
#                 dataDict[verblemma]['counts'][adpandcase] += 1
#                 formattedsent = build_sentence(sent.tokens, [currentindex, adpindex, caseindex])
#                 if formattedsent not in dataDict[verblemma]['sentences'][adpandcase]:
#                     dataDict[verblemma]['sentences'][adpandcase].append(formattedsent)
            possiblecx = set()
            if before == []:
                for af in after:
                    possiblecx.add(f'{verblemma} + {af}')
            elif after == []:
                for be in before:
                    possiblecx.add(f'{be} + {verblemma}')
            else:
                for be in before: 
                    for af in after:
                        possiblecx.add(f'{be} + {verblemma} + {af}')
                                   
            for cx in possiblecx:
                commonDict[verblemma].setdefault(cx, dict())
                commonDict[verblemma][cx].setdefault('counts',0)
                commonDict[verblemma][cx].setdefault('sentences',set())
                commonDict[verblemma][cx]['counts'] += 1
                commonDict[verblemma][cx]['sentences'].add(build_sentence(sent.tokens, [currentindex]))
        currentindex += 1

0/98892 sentences parsed...
10000/98892 sentences parsed...
20000/98892 sentences parsed...
30000/98892 sentences parsed...
40000/98892 sentences parsed...
50000/98892 sentences parsed...
60000/98892 sentences parsed...
70000/98892 sentences parsed...
80000/98892 sentences parsed...
90000/98892 sentences parsed...


In [43]:
commonDict['посадить'].keys()

dict_keys(['посадить + в/во + ACC', 'посадить + под + ACC', 'посадить + ACC', 'посадить + у + GEN', 'INST + посадить + на + ACC', 'INST + посадить + ACC', 'NOM + посадить + на + ACC', 'посадить + с/со + INST', 'ACC + посадить', 'NOM + посадить + ACC', 'посадить + на + ACC', 'ACC + посадить + в/во + ACC', 'ACC + посадить + ACC', 'на + ACC + посадить + ACC', 'между + INST + посадить + ACC', 'в/во + LOC + посадить + ACC', 'NOM + посадить', 'посадить + INST', 'между + INST + посадить', 'ACC + посадить + на + LOC', 'в/во + ACC + посадить', 'с/со + INST + посадить', 'посадить + для + GEN', 'ACC + посадить + с/со + INST'])

In [104]:
jsonDict = dict()
for verb in commonDict:
    cxlst = []
    for cx in commonDict[verb]:
        cxlst.append((commonDict[verb][cx]['counts'], cx)) 
        commonDict[verb][cx]['sentences'] = list(commonDict[verb][cx]['sentences'])
    sortedlist = sorted(cxlst, reverse=True)
    top3 = sortedlist[:3]
    jsonDict[verb] = []
    for tup in top3:
        jsonDict[verb].append({
            'cx' : tup[1],
            'data' : commonDict[verb][tup[1]]
        })
jsonList = [jsonDict]

In [105]:
pwd

'/Users/paigelee/Desktop/spring2021/clancy/verbhistograms'

In [106]:
import json
with open(f'common_constructions.json', 'w', encoding='utf8') as jsonfile:
    json.dump(jsonList, jsonfile, ensure_ascii=False)

In [97]:
### constructions 2

In [141]:
def build_sentence(tokens, upperlist):
    sentence = ''
    for i, token in enumerate(tokens):
        if token.pos == 'PUNCT':
            space = ''
        else:
            space = ' '
        if i in upperlist:
            text = token.text.upper()
        else:
            text = token.text
        if text == None:
            text = ''
        sentence = sentence + space + text
    return sentence.strip()
adpdict = {
    'в' : 'в/во',
    'во' : 'в/во',
    'с' : 'с/со',
    'со' : 'с/со',
    'о' : 'о/об',
    'об' : 'о/об',
    'обо' : 'о/об'
}
# sample = random.sample(rnccorpus,1000)
# sample = rnccorpus[35557:35558]
sample = rnccorpus

commonDict = dict()
for i, sent in enumerate(sample):
    if i % 10000 == 0:
        print(f'{i}/{len(sample)} sentences parsed...')
    currentindex = 0
    for token in sent.tokens:
        # for each verb found
        if token.pos == 'VERB':
            verblemma = token.lemma
            commonDict.setdefault(verblemma, dict())
            allcxx = []
            for window in range(-3,4):
                cxfound = False
                adpindex = currentindex + window

                if adpindex >= 0 and adpindex < len(sent.tokens):
                    adptoken = sent.tokens[adpindex]
                    if adptoken.pos == 'PUNCT' and adpindex + 1 >= 0 and adpindex + 1 < len(sent.tokens):
                        adpindex = adpindex + 1
                    if adptoken.pos == 'ADP':
                        adplemma = adptoken.lemma
                        # lemmatize prepositions
                        if adplemma in adpdict:
                            adplemma = adpdict[adplemma]
                        caseindex = adpindex + 1
                        if caseindex >= 0 and caseindex < len(sent.tokens):
                            casetoken = sent.tokens[caseindex]
                            if casetoken.pos == 'PUNCT' and caseindex + 1 >= 0 and caseindex + 1 < len(sent.tokens):
                                caseindex = caseindex + 1
                            casetoken = sent.tokens[caseindex]
                            try:
                                case = casetoken.feats['Case']
                                if case == 'Ins':
                                    case = 'Inst'
                                adpandcase = adplemma + ' + ' + case.upper()
                                cxfound = True    
                            except:
                                cxfound = False
                    else:
                        if window in [-1,1]:
                            # not an adposition directly before or after verb
                            try: 
                                if adpindex >= 0:
                                    # need to make sure prev token is not adp
                                    if sent.tokens[adpindex - 1].pos != 'ADP':
                                        case = adptoken.feats['Case']
                                        if case == 'Ins':
                                            case = 'Inst'
                                        adpandcase = case.upper()
                                        caseindex = currentindex
                                        cxfound = True
                            except:
                                # check if infinitive, if so, set caseindex to current index
                                if adptoken.pos == 'VERB':
                                    if 'VerbForm' in adptoken.feats and (adptoken.feats['VerbForm'] == 'Inf'):
                                        adpandcase = 'INFINITIVE'
                                        caseindex = currentindex
                                        cxfound = True
                if cxfound:
                    allcxx.append((adpandcase,adpindex,caseindex))

            for (cx, adpindex, caseindex) in allcxx:
                commonDict[verblemma].setdefault(cx, dict())
                commonDict[verblemma][cx].setdefault('counts',0)
                commonDict[verblemma][cx].setdefault('sentences',set())
                commonDict[verblemma][cx]['counts'] += 1
                commonDict[verblemma][cx]['sentences'].add(build_sentence(sent.tokens, [currentindex, adpindex, caseindex]))
                
        currentindex += 1

0/98892 sentences parsed...
10000/98892 sentences parsed...
20000/98892 sentences parsed...
30000/98892 sentences parsed...
40000/98892 sentences parsed...
50000/98892 sentences parsed...
60000/98892 sentences parsed...
70000/98892 sentences parsed...
80000/98892 sentences parsed...
90000/98892 sentences parsed...


In [142]:
commonDict['заботиться'].keys()

dict_keys(['о/об + LOC', 'NOM', 'в/во + ACC', 'INFINITIVE', 'в/во + LOC', 'при + LOC'])

In [174]:
# get highest numbered things / critical number
jsonDict = dict()
for verb in ['помочь']:
    cxlist = []
    for cx in commonDict[verb]:
        cxlist.append((commonDict[verb][cx]['counts'], cx))
    sortedlst = sorted(cxlist, reverse=True)
    newcx = verb
    print(sortedlst[:3])
    for ct, cx in sortedlst[:3]:
        if cx == 'NOM':
            newcx = cx + ' + ' + newcx
        else:
            newcx = newcx + ' + ' + cx
    if 'NOM' not in newcx:
        if 'NOM' in [cx for ct, cx in sortedlst[:5]]:
            newcx = 'NOM + ' + newcx
    jsonDict[verb] = newcx
jsonList = [jsonDict]

[(91, 'DAT'), (56, 'NOM'), (28, 'INFINITIVE')]


In [175]:
jsonDict['помочь']

'NOM + помочь + DAT + INFINITIVE'

In [176]:
import json
with open(f'common_constructions1.json', 'w', encoding='utf8') as jsonfile:
    json.dump(jsonList, jsonfile, ensure_ascii=False)